In [11]:
import pandas as pd
df = pd.read_table('virushostdb.daily.tsv')
df.columns.values


array(['virus tax id', 'virus name', 'virus lineage', 'refseq id',
       'KEGG GENOME', 'KEGG DISEASE', 'DISEASE', 'host tax id',
       'host name', 'host lineage', 'pmid', 'evidence', 'sample type',
       'source organism'], dtype=object)

筛选phage-host条目

In [12]:
df2 = df[['virus name', 'refseq id', 'evidence',
        'host name', 'host tax id', 'host lineage']].copy()
# df2[df2["host tax id"].isnull()]
# print(len(df2)) # 17959
# print(len(df2.dropna(axis=0, how='any'))) # 16537
df3 = df2.dropna(axis=0, how='any')

# df['host lineage'].str.split(';').apply(len)


def select_phage(x):
    try:
        kingdom = x.split(';')[0].strip()
        if kingdom == 'Bacteria' or kingdom == 'Archaea':
            return 1
        elif kingdom == 'Eukaryota':
            return 0
    except:
        print(x)


df_phage = df3[df3['host lineage'].apply(select_phage) == 1].copy()


In [13]:

df_phage['host tax id'] = df_phage['host tax id'].astype(int)
df_phage.to_csv('virushostdb.daily.phage.tsv', sep='\t', index=False)


发现24个phage有多个id

In [21]:

df_phage_multiple=df_phage[df_phage['refseq id'].apply(lambda x: ',' in x)]
df_phage_multiple.to_csv('virushostdb.daily.phage.multiple.tsv', sep='\t', index=False)

In [22]:
len(set(df_phage_multiple['virus name']))

24

检查后发现只有9个phage不完整，其他是只是同一个名称的不同测序结果，对不完整的phage进行添加GCF id

In [16]:
df_phage.loc[df_phage['virus name'] == 'Bacillus phage SPG24', 'refseq id']='GCF_001736955.2(NC_030903, NC_041835, NC_041836, NC_041837, NC_041838, NC_041839, NC_041840, NC_041841, NC_041842)'
df_phage.loc[df_phage['virus name'] == 'Pseudomonas phage phi12', 'refseq id']='GCF_000851005.1(NC_004173, NC_004174, NC_004175)'
df_phage.loc[df_phage['virus name'] == 'Pseudomonas phage phi13','refseq id'] = 'GCF_000852445.1(NC_004170, NC_004171, NC_004172)'
df_phage.loc[df_phage['virus name'] == 'Pseudomonas phage phi2954','refseq id'] = 'GCF_000882035.1(NC_012091, NC_012092, NC_012093)'
df_phage.loc[df_phage['virus name'] == 'Pseudomonas phage phi8','refseq id'] = 'GCF_000848645.1(NC_003299, NC_003300, NC_003301)'
df_phage.loc[df_phage['virus name'] == 'Pseudomonas phage phiNN','refseq id'] = 'GCF_002925585.1(NC_042068, NC_042069, NC_042070)'
df_phage.loc[df_phage['virus name'] == 'Pseudomonas phage phiYY','refseq id'] = 'GCF_002925595.1(NC_042071, NC_042072, NC_042073)'
df_phage.loc[df_phage['virus name'] == 'Pseudomonas virus phi6','refseq id'] = 'GCF_000852125.1(NC_003714, NC_003715, NC_003716)'
df_phage.loc[df_phage['virus name'] == 'Salmonella phage SP069','refseq id'] = 'GCF_002831045.1(NC_042062 NC_042063,NC_042064)'


VHDB phage name有重复的是因为记录了多个host

In [17]:
df_phage['virus name'].value_counts()


Salmonella phage OSY-STA                14
Enterobacteria phage PRD1                8
Bacillus phage vB_BsuM-Goe3              8
Bacillus phage SerPounce                 7
Bacillus phage BPS10C                    6
                                        ..
Bacteriophage DSS3_VP1                   1
Rhizobium phage vB_RleM_P10VF            1
Bacillus phage vB_BhaS-171               1
Gordonia phage Stultus                   1
Pseudomonas phage vB_PaeM_C2-10_Ab08     1
Name: virus name, Length: 4791, dtype: int64

## 添加 tax lineage信息
根据taxid生成tax lineage信息，只保留界门纲目科属种，如果不在有缺失则用上一级+unclassified
```bash
cut -f 5 virushostdb.daily.phage.tsv > taxid.tsv
cat taxid.tsv | taxonkit lineage | taxonkit reformat -f "{k}\t{p}\t{c}\t{o}\t{f}\t{g}\t{s}\t{t}" -P  | cut -f 3- |  tr '\t' ';' | tee lineage.tsv

cat taxid.tsv| taxonkit lineage -r -L | tee rank.tsv


paste rank.tsv lineage.tsv | tail -n+2 > host_info.tsv
```

In [18]:

df_info=pd.read_table('host_info.tsv',header=None,names=['host_tax_id','host_rank','host_lineage'])
df_info

,host_tax_id,host_rank,host_lineage
0,2148,species,k__Bacteria;p__Tenericutes;c__Mollicutes;o__Ac...
1,2148,species,k__Bacteria;p__Tenericutes;c__Mollicutes;o__Ac...
2,85698,species,k__Bacteria;p__Proteobacteria;c__Betaproteobac...
3,85698,species,k__Bacteria;p__Proteobacteria;c__Betaproteobac...
4,85698,species,k__Bacteria;p__Proteobacteria;c__Betaproteobac...
...,...,...,...
5199,632,species,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...
5200,632,species,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...
5201,632,species,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...
5202,632,species,k__Bacteria;p__Proteobacteria;c__Gammaproteoba...


In [19]:
df_phage['host_rank'] = df_info['host_rank'].to_list()

df_phage['common_lineage'] = df_info['host_lineage'].to_list()
df_phage=df_phage.reset_index(drop=True)
df_phage.to_csv('virushostdb.daily.phage.tsv', sep='\t', index=False)


##  统计信息

In [20]:
import pandas as pd

df=pd.read_table('virushostdb.daily.phage.tsv')
print(len(df))
print(len(set(df['host name'])))
print(len(set(df['virus name'])))

5204
841
4791
